In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the MongoDB data access object
from mongoDataAccessObject import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "abc9870"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.getAllAnimals())

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App front end
app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Show Logo Image
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        height='100px',
        width='100px'),
    html.P('Eyoel Tesfu. 10/11/2023'),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    
    
    html.Div([
        html.P('All Preferred Animals For: '),
        dcc.RadioItems(
            id='preferred-breed-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster/Individual Rescue', 'value': 'Disaster/Individual Rescue'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
        )
    ]),
    html.Hr(),
    html.P('(Reset the above to use this filter)'),
    
    # Use flexbox for side by side layout
    html.Div(style={'display': 'flex'}, children=[
        
        # Filter by animal type
        html.Div([
            html.P('Animal Type: '),
            dcc.Checklist(
                id='animal-type-filter',
                # distinct list of all animal types (bird, dog, cat, ...)
                options=[{'label': animal, 'value': animal} for animal in db.getAllProperties('animal_type')],
            )
        ]),
        # Filter by breed
        html.Div(style={'marginLeft':'50px'}, children=[
            html.P('Breed: '),
            dcc.Dropdown(
                id='animal-breed-filter',
                # distinct list of all breeds
                options=[{'label': breed, 'value': breed} for breed in db.getAllProperties('breed')],
                style={'width':'500px'},
            )
        ])

    ]),
    html.Hr(),
    # The Data Table
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),

        filter_action = 'native', 
        sort_action = 'native', 
        sort_mode = 'multi', 
        page_size = 10, 
        page_action = 'native', 
        row_selectable = 'single', 
        selected_columns=[],
        selected_rows=[0],
    ),
    
    # Users can select their desired number of rows displayed
    html.Div([
            html.Label("Rows per page:"),
            # Parameters and Arguments of the dropdown
            dcc.Dropdown(
                id='page-size-dropdown',
                options=[
                    {'label': '5 rows', 'value': 5},
                    {'label': '10 rows', 'value': 10},
                    {'label': '20 rows', 'value': 20},
                    {'label': '50 rows', 'value': 50},
                    {'label': '100 rows', 'value': 100}
                ],
                value=10,  # Default shown value
                clearable=False,
                style={'width':'100px'} # Width of the pagination size dropdown appearance
            )
        ]),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ])

])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to control how many rows are displayed in the datatable
@app.callback(
    Output('datatable-id', 'page_size'),
    [Input('page-size-dropdown', 'value')]
)
def update_page_size(value):
    return value
    
    
# Callback to filter the animals displayed in the datatable based on user input
@app.callback(Output('datatable-id','data'),
              [Input('animal-type-filter', 'value'),
               Input('animal-breed-filter', 'value'),
               Input('preferred-breed-filter', 'value')])
def update_dashboard(selected_types, selected_breed, preferred_breed):
    criteria = {}

    # Update criteria based on selected animal types
    if selected_types:
        criteria['animal_type'] = {'$in': selected_types}
    # Update criteria based on selected breed
    if selected_breed:
        criteria['breed'] = selected_breed

    # Modify criteria if a preferred breed is selected (not including the 'Reset' option)
    if preferred_breed != 'Reset':
        # Each preferred breed has specific criteria
        if preferred_breed == 'Water Rescue':
            criteria.update({
                'animal_type': 'Dog',
                'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                'sex_upon_outcome': 'Intact Female',
                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
            })
        elif preferred_breed == 'Mountain/Wilderness Rescue':
            criteria.update({
                'animal_type': 'Dog',
                'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                                  'Siberian Husky', 'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
            })
        elif preferred_breed == 'Disaster/Individual Rescue':
            criteria.update({
                'animal_type': 'Dog',
                'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound',
                                  'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
            })

    # Fetch the data using the criteria
    if criteria:
        filtered_df = pd.DataFrame.from_records(db.getAllAnimalsWithCriteria(criteria))
    else:
        filtered_df = pd.DataFrame.from_records(db.getAllAnimals())

    filtered_df.drop(columns=['_id'], inplace=True, errors='ignore')
    
    return filtered_df.to_dict('records')
    

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Convert the provided data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if 'breed' exists in dff
    if 'breed' not in dff.columns:
        return html.Div("No data available for visualization")

    # Calculate the count for each breed
    breed_counts = dff['breed'].value_counts()
    
    # Keep top N breeds and group others as 'Other'
    top_N = 8
    top_breeds = breed_counts.head(top_N).index
    dff['display_breed'] = np.where(dff['breed'].isin(top_breeds), dff['breed'], 'Other')
    
    # Create a pie chart
    fig = px.pie(dff, names='display_breed', title='Percentage of Animal Breeds in Shown Data', hole=0.3)
    # Enhance visual separation for top breeds
    fig.update_traces(textinfo='label+percent', pull=[0.1, 0.1, 0.1, 0.1, 0.2])  

    return [dcc.Graph(figure=fig)]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Ensure there's a selected row, and it's within range of the data.
    row = index[0] if index and 0 <= index[0] < len(dff) else None
    if row is None:
        return []
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Running the app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:21155/
